
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>

# 3.1 DEMO: Cross Cloud Replication with Cloudflare R2 [Recipient]

## Overview
This demo showcases how recipients can access replicated data from Cloudflare R2 and maintain synchronized local copies using `MERGE` operations for Type 1 Slowly Changing Dimensions (SCD). Recipients read changes from the R2-hosted external table and apply them to their local tables.

## Learning Objectives
By the end of this demo, you will understand:
1. How to access external tables hosted on Cloudflare R2
2. How to detect changes in replicated data
3. How to implement Type 1 SCD using `MERGE` operations
4. How to schedule automatic synchronization

We will use a Cloudflare R2 bucket as the storage location for an External Table which is used between the provider and recipients to store changes from a Managed source table, the changes are then replication asynchronously to another Managed table in the recipients region/provider.
<br />
<br />
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://github.com/stackql/databricks-data-sharing-and-collaboration/blob/main/images/cloudflare-r2-replication.png?raw=true"
    alt="Cloudflare R2 Replication"
  >
</div>
<br />
<br />

**Benefits:**
- Zero egress costs with Cloudflare R2
- Global data distribution without provider dependencies
- Automated change propagation using CDF
- Cost-effective sharing with unlimited recipients

## Setup

Run the common setup and demo configuration scripts.

In [ ]:
%run ./_common

In [ ]:
%run ./Demo-Setup-3_1

## Step 1: Create Storage Credential for R2 Access

Create storage credential to access the Cloudflare R2 bucket (read-only access).

In [ ]:
CREATE STORAGE CREDENTIAL IF NOT EXISTS r2_credential
WITH (
  AWS_ACCESS_KEY_ID 'your-r2-access-key',
  AWS_SECRET_ACCESS_KEY 'your-r2-secret-key'
)
COMMENT 'Cloudflare R2 credentials for reading replicated data'

## Step 2: Create External Location for R2

Create external location pointing to the R2 bucket where provider stores data.

In [ ]:
CREATE EXTERNAL LOCATION IF NOT EXISTS r2_location
URL 's3://databricks-demo/'
WITH (
  STORAGE_CREDENTIAL r2_credential
)
COMMENT 'Cloudflare R2 bucket for accessing replicated data'

## Step 3: Create External Table Reference to R2 Data

Create an external table that points to the provider's data on R2.

In [ ]:
CREATE OR REPLACE TABLE IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.r2_source_data')) (
  transaction_id STRING,
  customer_id STRING,
  product_category STRING,
  amount DECIMAL(10,2),
  transaction_date DATE,
  region STRING,
  created_at TIMESTAMP
)
LOCATION CONCAT('s3://databricks-demo/', DA.r2_path)
COMMENT 'External table pointing to provider data on Cloudflare R2'

## Step 4: Verify R2 Data Access

Check that we can successfully access the data from R2.

In [ ]:
-- Check data availability from R2
SELECT 
  COUNT(*) as total_records,
  MIN(transaction_date) as earliest_date,
  MAX(transaction_date) as latest_date,
  SUM(amount) as total_amount
FROM IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.r2_source_data'))

In [ ]:
-- Sample data from R2
SELECT * FROM IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.r2_source_data'))
ORDER BY created_at
LIMIT 5

## Step 5: Create Local Managed Table (Type 1 SCD)

Create a local managed table to store synchronized data with SCD metadata.

In [ ]:
CREATE OR REPLACE TABLE IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.local_transactions')) (
  transaction_id STRING,
  customer_id STRING,
  product_category STRING,
  amount DECIMAL(10,2),
  transaction_date DATE,
  region STRING,
  created_at TIMESTAMP,
  -- Type 1 SCD metadata
  last_updated_at TIMESTAMP,
  sync_timestamp TIMESTAMP
)
PARTITIONED BY (transaction_date)
TBLPROPERTIES (
  'delta.autoOptimize.optimizeWrite' = 'true',
  'delta.enableChangeDataFeed' = 'true'
)
COMMENT 'Local synchronized table with Type 1 SCD metadata'

## Step 6: Initial Data Load Using `MERGE`

Perform initial load from R2 to local table using `MERGE` operation.

In [ ]:
MERGE INTO IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.local_transactions')) AS target
USING (
  SELECT 
    transaction_id,
    customer_id,
    product_category,
    amount,
    transaction_date,
    region,
    created_at,
    current_timestamp() as last_updated_at,
    current_timestamp() as sync_timestamp
  FROM IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.r2_source_data'))
) AS source
ON target.transaction_id = source.transaction_id

WHEN MATCHED THEN UPDATE SET
  customer_id = source.customer_id,
  product_category = source.product_category,
  amount = source.amount,
  transaction_date = source.transaction_date,
  region = source.region,
  created_at = source.created_at,
  last_updated_at = source.last_updated_at,
  sync_timestamp = source.sync_timestamp

WHEN NOT MATCHED THEN INSERT (
  transaction_id, customer_id, product_category, amount,
  transaction_date, region, created_at, last_updated_at, sync_timestamp
) VALUES (
  source.transaction_id, source.customer_id, source.product_category,
  source.amount, source.transaction_date, source.region,
  source.created_at, source.last_updated_at, source.sync_timestamp
)

## Step 7: Verify Initial Synchronization

Check that the initial sync was successful.

In [ ]:
-- Compare record counts
SELECT 
  'R2 Source' as source,
  COUNT(*) as record_count,
  SUM(amount) as total_amount
FROM IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.r2_source_data'))

UNION ALL

SELECT 
  'Local Sync' as source,
  COUNT(*) as record_count,
  SUM(amount) as total_amount
FROM IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.local_transactions'))

In [ ]:
-- View synchronized data with SCD metadata
SELECT 
  transaction_id,
  amount,
  region,
  transaction_date,
  last_updated_at,
  sync_timestamp
FROM IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.local_transactions'))
ORDER BY sync_timestamp

## Step 8: Simulate Provider Adding New Data

Wait for the provider to add new data to R2, then refresh our external table to see changes.

**Note**: In a real scenario, you would monitor for changes or run this on a schedule.

In [ ]:
-- Refresh external table to pick up any new data from R2
REFRESH TABLE IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.r2_source_data'))

In [ ]:
-- Check for new data
SELECT COUNT(*) as current_r2_count 
FROM IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.r2_source_data'))

## Step 9: Detect and Sync Changes

Identify changes between R2 source and local table, then sync using `MERGE`.

In [ ]:
-- Identify new/changed records
SELECT 
  'New Records' as change_type,
  COUNT(*) as count
FROM IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.r2_source_data')) r2
LEFT ANTI JOIN IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.local_transactions')) local
ON r2.transaction_id = local.transaction_id

UNION ALL

SELECT 
  'Updated Records' as change_type,
  COUNT(*) as count
FROM IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.r2_source_data')) r2
INNER JOIN IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.local_transactions')) local
ON r2.transaction_id = local.transaction_id
WHERE r2.amount != local.amount 
   OR r2.region != local.region 
   OR r2.product_category != local.product_category

## Step 10: Apply Changes Using `MERGE` (Type 1 SCD)

Synchronize any detected changes using `MERGE` operation.

In [ ]:
MERGE INTO IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.local_transactions')) AS target
USING (
  SELECT 
    transaction_id,
    customer_id,
    product_category,
    amount,
    transaction_date,
    region,
    created_at,
    current_timestamp() as last_updated_at,
    current_timestamp() as sync_timestamp
  FROM IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.r2_source_data'))
) AS source
ON target.transaction_id = source.transaction_id

WHEN MATCHED THEN UPDATE SET
  customer_id = source.customer_id,
  product_category = source.product_category,
  amount = source.amount,
  transaction_date = source.transaction_date,
  region = source.region,
  created_at = source.created_at,
  last_updated_at = source.last_updated_at,
  sync_timestamp = source.sync_timestamp

WHEN NOT MATCHED THEN INSERT (
  transaction_id, customer_id, product_category, amount,
  transaction_date, region, created_at, last_updated_at, sync_timestamp
) VALUES (
  source.transaction_id, source.customer_id, source.product_category,
  source.amount, source.transaction_date, source.region,
  source.created_at, source.last_updated_at, source.sync_timestamp
)

## Step 11: Final Verification

Verify that synchronization is complete and data quality is maintained.

In [ ]:
-- Final sync verification
SELECT 
  'R2 Source' as table_name,
  COUNT(*) as total_records,
  SUM(amount) as total_amount,
  MAX(created_at) as latest_record
FROM IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.r2_source_data'))

UNION ALL

SELECT 
  'Local Synchronized' as table_name,
  COUNT(*) as total_records,
  SUM(amount) as total_amount,
  MAX(created_at) as latest_record
FROM IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.local_transactions'))

In [ ]:
-- View Type 1 SCD metadata
SELECT 
  transaction_id,
  amount,
  region,
  transaction_date,
  last_updated_at,
  sync_timestamp,
  CASE 
    WHEN last_updated_at > DATE_SUB(current_timestamp(), 1) THEN 'Recently Updated'
    ELSE 'Older Record'
  END as freshness_status
FROM IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.local_transactions'))
ORDER BY last_updated_at DESC

## Step 12: Create Reusable Sync View

Create a view that can be used to easily check sync status.

In [ ]:
CREATE OR REPLACE VIEW IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.sync_status')) AS
SELECT 
  r2.record_count as r2_records,
  local.record_count as local_records,
  CASE 
    WHEN r2.record_count = local.record_count THEN 'IN_SYNC'
    ELSE 'OUT_OF_SYNC'
  END as sync_status,
  r2.total_amount as r2_total_amount,
  local.total_amount as local_total_amount,
  local.last_sync,
  current_timestamp() as check_time
FROM (
  SELECT 
    COUNT(*) as record_count,
    SUM(amount) as total_amount
  FROM IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.r2_source_data'))
) r2
CROSS JOIN (
  SELECT 
    COUNT(*) as record_count,
    SUM(amount) as total_amount,
    MAX(sync_timestamp) as last_sync
  FROM IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.local_transactions'))
) local

In [ ]:
-- Check current sync status
SELECT * FROM IDENTIFIER(CONCAT(DA.catalog, '.', DA.schema, '.sync_status'))

## Summary

### What We Accomplished:

✅ **R2 Access**: Connected to Cloudflare R2 external table with zero egress costs  
✅ **Local Table**: Created managed table with Type 1 SCD metadata  
✅ **Change Detection**: Identified new and updated records from R2 source  
✅ **MERGE Operations**: Used `MERGE` for efficient Type 1 SCD synchronization  
✅ **Data Quality**: Verified synchronization accuracy and completeness  
✅ **Monitoring**: Created sync status view for ongoing monitoring  

### Type 1 SCD Implementation:

Our Type 1 SCD approach provides:
- **Current Data Only**: Overwrites old values with new ones (no history)
- **Metadata Tracking**: `last_updated_at` and `sync_timestamp` for auditing
- **`MERGE` Logic**: Efficient upsert operations (`UPDATE` existing, `INSERT` new)
- **Change Detection**: Automatic identification of modifications

### Key Benefits:

**Global Access**: Fast data access from Cloudflare's global network  
**Cost Efficient**: Zero egress fees for reading from R2  
**Local Performance**: Optimized local queries on synchronized data  
**Automatic Updates**: MERGE-based synchronization maintains current data  
**Data Quality**: Built-in validation and monitoring capabilities  

### Next Steps for Production:

1. **Automation**: Schedule the `MERGE` operation as a Databricks Job (hourly/daily)
2. **Secrets**: Use Databricks Secrets for R2 credentials
3. **Monitoring**: Set up alerts for sync failures or data quality issues
4. **Optimization**: Add partition pruning and table optimization
5. **Advanced SCD**: Consider Type 2 SCD if historical tracking is needed

The recipient now has a robust, cost-effective way to stay synchronized with provider data!

---
&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>